In [320]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as scs
from math import  inf
from docplex.mp.model import Model
#parameters
yields={1:[3,2.5,2,3,2.5,2,3,2.5,2] , 2:[3.6,3,2.4,3.6,3,2.4,3.6,3,2.4] , 3:[24,20,16,24,20,16,24,20,16]}
probability={1:1/3 , 2:1/3 , 3:1/3}
plant=[150,230,260]
buy =[238,210]
sell=[170, 150, 36, 10]
#define models
for x in range(1,4):
    exec("NLDS2" + str(x) + " = Model(name='NLDS2{}'.format(x))")
NLDS2=[NLDS21,NLDS22,NLDS23]
for x in range(1,10):
    exec("NLDS3" + str(x) + " = Model(name='NLDS3{}'.format(x))")
NLDS3=[NLDS31,NLDS32,NLDS33,NLDS34,NLDS35,NLDS36,NLDS37,NLDS38,NLDS39]
#define E21 and e21
E21=np.zeros([5,3])
e21=np.zeros(5)
E11=np.zeros([5,3])
e11=np.zeros(5)

In [319]:
i=1
##########################
#######################################FORWARD DIRECTION####################################################################################
############################################################################################################################################
while i<=10:
    print('###########################################ITERATION{}#######################################.'.format(i))
    NLDS1=Model(name='NLDS1')
    x1=NLDS1.continuous_var_matrix(range(1,4),range(1,3),name='x',key_format='%s')
    teta1=NLDS1.continuous_var_list(range(1,2),lb=-inf,name='teta1',key_format='%s')
    NLDS1.add_constraint_(x1[1,1]+x1[2,1]+x1[3,1]<=500)
    if i==1:
        NLDS1.add_constraint_(teta1[0]==0,'g')
    if i>1:
        for k in range(i-1):
            NLDS1.add_constraint_(x1[1,1] * E11[k][0] + x1[2,1] * E11[k][1] + x1[3,1] * E11[k][2] + teta1[0] >= e11[k])

    NLDS1.minimize(sum(plant[i-1]*x1[i,j] for i in range(1,4) for j in range(1,2))+teta1[0])
    sol_NLDS1=NLDS1.solve()
    sol_NLDS1.display()
    pi_NLDS2={}
    for s in range(1,4):
        NLDS2[s-1]=Model(name='NLDS2{}'.format(s))
        #variables
        x2=NLDS2[s-1].continuous_var_matrix(range(1,4),range(1,3),name='x',key_format='%s')
        teta2=NLDS2[s-1].continuous_var_list(range(1,4),lb=-inf,name='teta2',key_format='%s')
        y2=NLDS2[s-1].continuous_var_cube(range(1,3),range(1,4),range(2,4),name='y',key_format='%s')
        w2=NLDS2[s-1].continuous_var_cube(range(1,5),range(1,4),range(2,4),name='w',key_format='%s')
        #constraints
        NLDS2[s-1].add_constraint_(-x2[1,2]-x2[2,2]-x2[3,2]>=-500,'a')
        NLDS2[s-1].add_constraint_(sol_NLDS1[x1[1,1]] * yields[1][s-1] + y2[1,s,2] - w2[1,s,2] >= 200,'b')
        NLDS2[s-1].add_constraint_(sol_NLDS1[x1[2,1]] * yields[2][s-1] + y2[2,s,2] - w2[2,s,2] >= 240,'c')
        NLDS2[s-1].add_constraint_(sol_NLDS1[x1[3,1]] * yields[3][s-1] - w2[3,s,2] - w2[4,s,2] >= 0,'d')
        NLDS2[s-1].add_constraint_(-w2[3,s,2]>=-6000,'e')
        NLDS2[s-1].add_constraint_(sol_NLDS1[x1[1,1]] + sol_NLDS1[x1[2,1]] - x2[3,2]>=0,'f')
        if i==1:
                NLDS2[s-1].add_constraint_(teta2[s-1]==0,'g')
        if i>1:
            for k in range(i-1):
                NLDS2[s-1].add_constraint_(E21[k][0] * x2[1,2] + E21[k][1] * x2[2,2] + E21[k][2] * x2[3,2] + teta2[s-1] >= e21[k])
    
        #objective function
        NLDS2[s-1].minimize(150*x2[1,2]+230*x2[2,2]+260*x2[3,2]+sum(buy[i-1]*y2[i,s,2] for i in range(1,3))+
                    sum(sell[j-1]*w2[j,s,2] for j in range(1,5))+teta2[s-1])   
        sol_NLDS2=NLDS2[s-1].solve()
        sol_NLDS2.display()
        simplex=[]
        simplex.append(np.array(NLDS2[s-1].dual_values(NLDS2[s-1].find_matching_linear_constraints('a'))).item())
        simplex.append(np.array(NLDS2[s-1].dual_values(NLDS2[s-1].find_matching_linear_constraints('b'))).item())
        simplex.append(np.array(NLDS2[s-1].dual_values(NLDS2[s-1].find_matching_linear_constraints('c'))).item())
        simplex.append(np.array(NLDS2[s-1].dual_values(NLDS2[s-1].find_matching_linear_constraints('d'))).item())
        simplex.append(np.array(NLDS2[s-1].dual_values(NLDS2[s-1].find_matching_linear_constraints('e'))).item())
        simplex.append(np.array(NLDS2[s-1].dual_values(NLDS2[s-1].find_matching_linear_constraints('f'))).item())
        pi_NLDS2['NLDS2{}'.format(s)]=simplex
    print('simplex multipliers for NLDS2',pi_NLDS2)
    pi_NLDS3={}
    for s in range(1,10):
        NLDS3[s-1]=Model(name='NLDS3{}'.format(s))
        #variables
        teta3=NLDS3[s-1].continuous_var_list(range(1,10),lb=-inf,name='teta2',key_format='%s')
        y3=NLDS3[s-1].continuous_var_cube(range(1,3),range(1,10),range(2,4),name='y',key_format='%s')
        w3=NLDS3[s-1].continuous_var_cube(range(1,5),range(1,10),range(2,4),name='w',key_format='%s')
        #constraints
        NLDS3[s-1].add_constraint_(sol_NLDS2[x2[1,2]] * yields[1][s-1] + y3[1,s,3] - w3[1,s,2] >= 200,'a')
        NLDS3[s-1].add_constraint_(sol_NLDS2[x2[2,2]] * yields[2][s-1] + y3[2,s,3] - w3[2,s,3] >= 240,'b')
        NLDS3[s-1].add_constraint_(sol_NLDS2[x2[3,2]] * yields[3][s-1] - w3[3,s,3] - w3[4,s,3] >= 0,'c')
        NLDS3[s-1].add_constraint_(-w3[3,s,3]>=-6000,'d')
        NLDS3[s-1].add_constraint_(teta3[s-1]==0)
        #objective function
        NLDS3[s-1].minimize(sum(buy[i-1]*y3[i,s,3] for i in range(1,3))+
                    sum(sell[j-1]*w3[j,s,3] for j in range(1,5))+teta3[s-1])
        sol_NLDS3=NLDS3[s-1].solve()
        NLDS3_sol.append(sol_NLDS3)
        sol_NLDS3.display()
        simplex=[]
        simplex.append(np.array(NLDS3[s-1].dual_values(NLDS3[s-1].find_matching_linear_constraints('a'))).item())
        simplex.append(np.array(NLDS3[s-1].dual_values(NLDS3[s-1].find_matching_linear_constraints('b'))).item())
        simplex.append(np.array(NLDS3[s-1].dual_values(NLDS3[s-1].find_matching_linear_constraints('c'))).item())
        simplex.append(np.array(NLDS3[s-1].dual_values(NLDS3[s-1].find_matching_linear_constraints('d'))).item())
        pi_NLDS3['NLDS3{}'.format(s)]=simplex
    print('simplex multipliers for NLDS3',pi_NLDS3)
    ##########################################################################################################################
    ############################################### BACKWARD DIRECTION########################################################
    ##########################################################################################################################
    #t-1=2
    T21=np.array([[3,0,0],[0,3.6,0],[0,0,24],[0,0,0]])
    T22=np.array([[2.5,0,0],[0,3,0],[0,0,20],[0,0,0]])
    T23=np.array([[2,0,0],[0,2.4,0],[0,0,16],[0,0,0]])
    pi31=np.array(pi_NLDS3['NLDS31']).reshape(1,4)
    pi32=np.array(pi_NLDS3['NLDS32']).reshape(1,4)
    pi33=np.array(pi_NLDS3['NLDS33']).reshape(1,4)
    E21[i-1]=(1/3)*np.matmul(pi31,T21)+(1/3)*np.matmul(pi32,T22)+(1/3)*np.matmul(pi33,T23)
    print('E21:',E21[i-1])
    #E22 and E23 will be the same as E21
    h31=np.array([200,240,0,-6000]).reshape(4,1)
    h32=h31
    h33=h31
    e21[i-1]=(1/3)*np.matmul(pi31,h31)+(1/3)*np.matmul(pi32,h32)+(1/3)*np.matmul(pi33,h33)
    print('e21:',e21[i-1])
    x21=np.array([sol_NLDS2[x2[1,2]] ,sol_NLDS2[x2[2,2]] ,
    sol_NLDS2[x2[3,2]]]).reshape(3,1)
    if e21[i-1]-np.matmul(E21[i-1],x21)>=sol_NLDS2[teta2[0]]:
        print('condition does not hold - we add a cut to NLDS21 problem')
    else:
        print('no cut needed go to t-1')
    #########################################
    for s in range(1,4):
        NLDS2[s-1]=Model(name='NLDS2{}'.format(s))
        #variables
        x2=NLDS2[s-1].continuous_var_matrix(range(1,4),range(1,3),name='x',key_format='%s')
        teta2=NLDS2[s-1].continuous_var_list(range(1,4),lb=-inf,name='teta2',key_format='%s')
        y2=NLDS2[s-1].continuous_var_cube(range(1,3),range(1,4),range(2,4),name='y',key_format='%s')
        w2=NLDS2[s-1].continuous_var_cube(range(1,5),range(1,4),range(2,4),name='w',key_format='%s')
        #constraints
        NLDS2[s-1].add_constraint_(-x2[1,2]-x2[2,2]-x2[3,2]>=-500,'a')
        NLDS2[s-1].add_constraint_(sol_NLDS1[x1[1,1]] * yields[1][s-1] + y2[1,s,2] - w2[1,s,2] >= 200,'b')
        NLDS2[s-1].add_constraint_(sol_NLDS1[x1[2,1]] * yields[2][s-1] + y2[2,s,2] - w2[2,s,2] >= 240,'c')
        NLDS2[s-1].add_constraint_(sol_NLDS1[x1[3,1]] * yields[3][s-1] - w2[3,s,2] - w2[4,s,2] >= 0,'d')
        NLDS2[s-1].add_constraint_(-w2[3,s,2]>=-6000,'e')
        NLDS2[s-1].add_constraint_(sol_NLDS1[x1[1,1]] + sol_NLDS1[x1[2,1]] - x2[3,2]>=0,'f')
        for x in range(i):
                NLDS2[s-1].add_constraint_(E21[x][0] * x2[1,2] + E21[x][1] * x2[2,2] + E21[x][2] * x2[3,2] + teta2[s-1] >= e21[x])
        #objective function
        NLDS2[s-1].minimize(150*x2[1,2]+230*x2[2,2]+260*x2[3,2]+sum(buy[i-1]*y2[i,s,2] for i in range(1,3))+
                    sum(sell[j-1]*w2[j,s,2] for j in range(1,5))+teta2[s-1])
        NLDS2[s-1].solve().display()
        simplex=[]
        simplex.append(np.array(NLDS2[s-1].dual_values(NLDS2[s-1].find_matching_linear_constraints('a'))).item())
        simplex.append(np.array(NLDS2[s-1].dual_values(NLDS2[s-1].find_matching_linear_constraints('b'))).item())
        simplex.append(np.array(NLDS2[s-1].dual_values(NLDS2[s-1].find_matching_linear_constraints('c'))).item())
        simplex.append(np.array(NLDS2[s-1].dual_values(NLDS2[s-1].find_matching_linear_constraints('d'))).item())
        simplex.append(np.array(NLDS2[s-1].dual_values(NLDS2[s-1].find_matching_linear_constraints('e'))).item())
        simplex.append(np.array(NLDS2[s-1].dual_values(NLDS2[s-1].find_matching_linear_constraints('f'))).item())
        pi_NLDS2['NLDS2{}'.format(s)]=simplex
    #t-1=1
    T11=np.array([[3,0,0],[0,3.6,0],[0,0,24],[0,0,0],[0,0,0],[0,0,0]])
    T12=np.array([[2.5,0,0],[0,3,0],[0,0,20],[0,0,0],[0,0,0],[0,0,0]])
    T13=np.array([[2,0,0],[0,2.4,0],[0,0,16],[0,0,0],[0,0,0],[0,0,0]])
    pi21=np.array(pi_NLDS2['NLDS21']).reshape(1,6)
    pi22=np.array(pi_NLDS2['NLDS22']).reshape(1,6)
    pi23=np.array(pi_NLDS2['NLDS23']).reshape(1,6)
    E11[i-1]=(1/3)*np.matmul(pi21,T11)+(1/3)*np.matmul(pi22,T12)+(1/3)*np.matmul(pi23,T13)
    print('E11:',E11[i-1])
    #E22 and E23 will be the same as E21
    h21=np.array([-500,200,240,0,-6000,0]).reshape(6,1)
    h22=h21
    h23=h21
    e11[i-1]=(1/3)*np.matmul(pi21,h21)+(1/3)*np.matmul(pi22,h22)+(1/3)*np.matmul(pi23,h23)
    print('e11:',e11[i-1])
    x11=np.array([sol_NLDS1[x1[1,1]] ,sol_NLDS1[x1[2,1]] ,
    sol_NLDS1[x1[3,1]]]).reshape(3,1)
    if e21[i-1]-np.matmul(E11[i-1],x11)>=sol_NLDS1.get_value_list(teta1)[0]:
        print('condition does not hold - we add a cut to NLDS1 problem')
    else:
        print('no cut needed - optimal solution achieved!')
        break
    NLDS1.add_constraint_(x1[1,1] * E11[i-1][0] + x1[2,1] * E11[i-1][1] + x1[3,1] * E11[i-1][2] + teta1[0] >= e11[i-1])
    NLDS1.remove_constraint('g')
    sol_NLDS1=NLDS1.solve()
    sol_NLDS1.display()



    i+=1





###########################################ITERATION1#######################################.
solution for: NLDS1
objective: 0.000
solution for: NLDS21
objective: 98000.000
y112 = 200.000
y212 = 240.000
solution for: NLDS22
objective: 98000.000
y122 = 200.000
y222 = 240.000
solution for: NLDS23
objective: 98000.000
y132 = 200.000
y232 = 240.000
simplex multipliers for NLDS2 {'NLDS21': [0, 238.0, 210.0, 0, 0, 0], 'NLDS22': [0, 238.0, 210.0, 0, 0, 0], 'NLDS23': [0, 238.0, 210.0, 0, 0, 0]}
solution for: NLDS31
objective: 98000.000
y113 = 200.000
y213 = 240.000
solution for: NLDS32
objective: 98000.000
y123 = 200.000
y223 = 240.000
solution for: NLDS33
objective: 98000.000
y133 = 200.000
y233 = 240.000
solution for: NLDS34
objective: 98000.000
y143 = 200.000
y243 = 240.000
solution for: NLDS35
objective: 98000.000
y153 = 200.000
y253 = 240.000
solution for: NLDS36
objective: 98000.000
y163 = 200.000
y263 = 240.000
solution for: NLDS37
objective: 98000.000
y173 = 200.000
y273 = 240.000
solu

In [310]:
kir=1
while kir<=4:
    print(kir)
    kir+=1
    if kir==3:
        print('ok')

1
2
ok
3
4


In [304]:
E11

array([[1112.5       ,  714.        , 4200.        ],
       [1000.        ,  714.        , 4200.        ],
       [ 122.34042553,  714.        , 4200.        ],
       [   0.        ,  714.        , 4200.        ],
       [   0.        ,  714.        , 4200.        ],
       [   0.        ,  714.        , 4200.        ],
       [   0.        ,  714.        , 4200.        ],
       [   0.        ,  714.        , 4200.        ],
       [   0.        ,  714.        , 4200.        ],
       [   0.        ,  714.        , 4200.        ]])

In [241]:
    NLDS1=Model(name='NLDS1')
    x1=NLDS1.continuous_var_matrix(range(1,4),range(1,3),name='x',key_format='%s')
    teta1=NLDS1.continuous_var_list(range(1,2),lb=-inf,name='teta1',key_format='%s')
    NLDS1.add_constraint_(x1[1,1]+x1[2,1]+x1[3,1]<=500)
    if i==1:
        NLDS1.add_constraint_(teta1[0]==0,'g')
    if i>1:
        for k in range(i-1):
            NLDS1.add_constraint_(E11[k][0] * x1[1,1]  + E11[k][1] * x1[2,1] + E11[k][2] * x1[3,1]  + teta1[0] >= e11[k])
    NLDS1.minimize(sum(plant[i-1]*x1[i,j] for i in range(1,4) for j in range(1,2))+teta1[0])
    sol_NLDS1=NLDS1.solve()
    sol_NLDS1.display()
            

solution for: NLDS1
objective: -2094500.000
x31 = 500.000
teta11 = -2224500.000
